In [ ]:
# Connecting to bitcoind: https://bitcoinlib.readthedocs.io/en/latest/source/_static/manuals.setup-bitcoind-connection.html

from bitcoinlib.services.bitcoind import BitcoindClient
import os

config_file = os.path.join(os.getcwd(), 'bitcoin.conf') 
bdc = BitcoindClient.from_config(config_file)
bdc.getinfo()

In [79]:
# Generating a seed 
# NOTE - seeds generated without a sufficient amount of entropy are not secure.
# bitcoinlib uses urandom which generally has a high amount of entropy but I would still not recommend the below method
# for generating seeds that are used to hold funds that you would not want to lose. Look into alternatives.
# Reddit thread discussing recent libbitcoin seed entropy issue: https://www.reddit.com/r/Buttcoin/comments/15n1nfe/be_your_own_bank_is_going_great_libbitcoin/

# https://bitcoinlib.readthedocs.io/en/latest/source/bitcoinlib.mnemonic.html#bitcoinlib.mnemonic.Mnemonic.generate
from bitcoinlib.mnemonic import Mnemonic
mnemonic = Mnemonic()

# Use strength 256 for a 24 word seed
seed = mnemonic.generate(strength=128)
print("This seed can be entered in another bitcoin wallet client application, such as Sparrow. Make sure to select testnet")
print(seed)
# seed = 'whatever'


In [ ]:
# Creating an HDKey using your seed
# https://bitcoinlib.readthedocs.io/en/latest/source/bitcoinlib.keys.html#bitcoinlib.keys.HDKey.from_passphrase
from bitcoinlib.keys import HDKey

key = HDKey.from_passphrase(seed, witness_type='segwit', network='testnet')
key.info()

In [ ]:
# Create a wallet
# https://bitcoinlib.readthedocs.io/en/latest/source/bitcoinlib.wallets.html#bitcoinlib.wallets.wallet_create_or_open
from bitcoinlib.wallets import wallet_create_or_open, wallets_list

# Bitcoinlib maintains wallets in a database on your computer. 
# By default this is a sqlite database located at ~/.bitcoinlib/database/bitcoinlib.sqlite
# wallets_list()
wallet = wallet_create_or_open('Charlotte Bitcoin Workshop', keys=[key], network='testnet')
wallet.info()

In [ ]:
db_keys = wallet.keys(depth=5)
first_key = db_keys[0]
addresses = [key.address for key in db_keys]
addresses

In [54]:
new_key = wallet.new_key()

In [ ]:
first_key.__dict__

In [ ]:
# Go to a bitcoin testnet faucet and get some bitcoin sent to your first address
# https://testnet-faucet.com/btc-testnet/
# https://coinfaucet.eu/en/btc-testnet/
# https://bitcoinfaucet.uo1.net/
# https://tbtc.bitaps.com/
# https://kuttler.eu/en/bitcoin/btc/faucet/
first_key.address

In [ ]:
wallet.balance_update_from_serviceprovider()

In [62]:
wallet.transactions_update()
transactions = wallet.transactions()
first_transaction = transactions[0]

In [ ]:
first_transaction.as_dict()

In [ ]:
# Parsing transactions
# https://bitcoinlib.readthedocs.io/en/latest/source/bitcoinlib.transactions.html#bitcoinlib.transactions.Transaction.parse
from bitcoinlib.transactions import Transaction, transaction_deserialize

# Find a random txid off of https://mempool.space/testnet
txid = '4695f3554532d8194873de5e1b89a773e418d194f5b837cd20fd3f4b90140046'
rawtx = bdc.getrawtransaction(txid)
#print(rawtx)
t = Transaction.parse(rawtx)
t.info()

In [90]:
other_wallet = wallet_create_or_open('Testnet Wallet')

In [ ]:
#other_wallet.balance_update_from_serviceprovider()
other_wallet.scan()
other_wallet.balance()

In [98]:
second_key = db_keys[1]
other_t = other_wallet.send_to(second_key.address, 500, offline=False)

In [ ]:
other_wallet.transactions_update()
other_ts = other_wallet.transactions()
print([o.date for o in other_ts])

In [ ]:
other_t.info()

In [ ]:
other_wallet.new_key()

In [ ]:
other_keys = other_wallet.keys(depth=5, has_balance=False)
addresses = [key.address for key in other_keys]
addresses

In [131]:
new_t_to_other = wallet.transaction_create([('tb1qwx5y0dv02w040x2xqav6dccl5h3fqdpl6uvpl8', 1000)])

In [146]:
#new_t_to_other.send()
new_t_to_other.sign(key)

In [ ]:
bdc.sendrawtransaction(new_t_to_other.raw_hex())